# Consumer, Product, and Sales Analysis KPIs

The Store is a struggling business specializing in scale model cars that has been hit hard by supply chain delays and inventory management issues. As such, many popular products are frequently under-stocked, turning away customers that otherwise would have spent more. The goal of this data analysis project is to use SQL to find KPIs relating to product inventory, consumer behaviour, and sales strategies. Specifically, I aim to answer the following questions:

1. Which products are highest-performing (sell the most) yet are constantly in low stock?
2. Who are the highest- and lowest-spending customers and how can they be categorized into spending brackets?
3. What is the average profit that each customer brings in that can be re-spent and invested into the store?

I conclude that the Classic Cars product line make up the majority of the top ten most in-demand and low-stock products in the Store, meaning they should be re-stocked most often to optimize inventory needs. The highest- and lowest-spending customers tend to vary with no clear pattern, and the average customer brings in $39,039.59 over their lifetime in the Store. These findings may be used to help the Store determine how it can better optimize
its inventory, reward loyal customers, and wisely calculate how its available funds from lifetime customer profits can be used to attract new customers.

### Loading in the stores.db database

I first load in the database for analysis with SQL.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///stores.db
    

### The Database

I begin by exploring the database and summarizing the results into a table consisting of each table's name, counted number of attributes, and number of rows. A sample of the first five rows of the `customers` table is then displayed.

In [2]:
%%sql

SELECT 'Customers' AS table_name,
       '13' AS number_of_attributes,
       COUNT(*) AS number_of_rows
  FROM customers

 UNION ALL
    
SELECT 'Employees' AS table_name,
       '8' AS number_of_attributes,
       COUNT(*) AS number_of_rows
  FROM employees

 UNION ALL

SELECT 'Offices' AS table_name,
       '9' AS number_of_attributes,
       COUNT(*) AS number_of_rows
  FROM offices

 UNION ALL

SELECT 'OrderDetails' AS table_name,
       '5' AS number_of_attributes,
       COUNT(*) AS number_of_rows
  FROM orderdetails
  
 UNION ALL

SELECT 'Orders' AS table_name,
       '7' AS number_of_attributes,
       COUNT(*) AS number_of_rows
  FROM orders
  
 UNION ALL

SELECT 'Payments' AS table_name,
       '4' AS number_of_attributes,
       COUNT(*) AS number_of_rows
  FROM payments
  
 UNION ALL

SELECT 'ProductLines' AS table_name,
       '4' AS number_of_attributes,
       COUNT(*) AS number_of_rows
  FROM productlines
  
 UNION ALL

SELECT 'Products' AS table_name,
       '9' AS number_of_attributes,
       COUNT(*) AS number_of_rows
  FROM products


 * sqlite:///stores.db
Done.


table_name,number_of_attributes,number_of_rows
Customers,13,122
Employees,8,23
Offices,9,7
OrderDetails,5,2996
Orders,7,326
Payments,4,273
ProductLines,4,7
Products,9,110


In [3]:
%%sql

SELECT *
  FROM customers
 LIMIT 5;


 * sqlite:///stores.db
Done.


customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370,21000
112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166,71800
114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611,117300
119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,France,1370,118200
121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504,81700


The database consists of 8 different tables representing various operations of the Store, including its customers, orders and sales, and products. After examining the database closer, I see the `orders` and `orderdetails` tables share an `orderNumber` attribute, and the `orderdetails` and `products` tables share a `productCode` attribute. This will be useful for joining tables later on.

### Product Optimization Analysis: Determining Priority Products

To answer the first question and determine which products are highest in demand but low in stock, I calculate a ratio dividing a product's sum of all quantities ordered by the quantity in stock; the greater the ratio, the more a product is in demand without sufficient stock. Ratios of greater than 1 would represent products with more orders than inventory available.

In [4]:
%%sql

SELECT productCode,
       ROUND((SELECT SUM(quantityOrdered) * 1.0
               FROM orderdetails AS od
               WHERE od.productCode = p.productCode) / p.quantityInStock, 2) AS low_stock
  FROM products AS p
 GROUP BY productCode
 ORDER BY low_stock DESC
 LIMIT 10;

 * sqlite:///stores.db
Done.


productCode,low_stock
S24_2000,67.67
S12_1099,13.72
S32_4289,7.15
S32_1374,5.7
S72_3212,2.31
S700_3167,1.9
S50_4713,1.65
S18_2795,1.61
S18_2248,1.54
S700_1938,1.22


To supplement the low-stock ratio, I can also quantify the performance of each of the Store's products by determining the total revenue each product brings in: multiplying the quantity ordered by the sale price and summing it up for all orders of the product.

In [5]:
%%sql

SELECT productCode, SUM(quantityOrdered * priceEach) AS product_performance
  FROM orderdetails
 GROUP BY productCode
 ORDER BY product_performance DESC
 LIMIT 10;

 * sqlite:///stores.db
Done.


productCode,product_performance
S18_3232,276839.98
S12_1108,190755.86
S10_1949,190017.95999999996
S10_4698,170685.99999999997
S12_1099,161531.47999999992
S12_3891,152543.02
S18_1662,144959.90999999997
S18_2238,142530.62999999998
S18_1749,140535.60000000003
S12_2823,135767.03000000003


The highest-priority products are those with high performance *and* a high low stock ratio. I can determine this by converting the product performance calculation for products into a CTS, tabulating a priority index by multiplying the product performance and low stock calculations (a higher low stock ratio means the demand is high and inventory is low), and sorting the final products based on the priority index.

In [6]:
%%sql

WITH
product_performance_table AS (
    SELECT productCode, SUM(quantityOrdered * priceEach) AS product_performance
      FROM orderdetails
     GROUP BY productCode
     ORDER BY product_performance DESC
     LIMIT 10
)

SELECT p.productCode AS priority_product_codes, p.productLine,
        
        -- multiply the low stock ratio by the product performance to quantify a priority level index
    
        ROUND(((SELECT SUM(quantityOrdered) * 1.0
                  FROM orderdetails AS od
                 WHERE od.productCode = p.productCode) / p.quantityInStock) * ppt.product_performance
             ) AS priority_index
          
  FROM products AS p
  JOIN product_performance_table AS ppt
    ON p.productCode = ppt.productCode
 WHERE p.productCode IN (SELECT productCode
                           FROM product_performance_table)
 GROUP BY p.productCode
 ORDER BY priority_index DESC
 LIMIT 10;


 * sqlite:///stores.db
Done.


priority_product_codes,productLine,priority_index
S12_1099,Classic Cars,2216307.0
S12_3891,Classic Cars,140328.0
S18_3232,Classic Cars,59965.0
S12_1108,Classic Cars,53711.0
S18_1749,Vintage Cars,47361.0
S10_4698,Motorcycles,30119.0
S18_2238,Classic Cars,29749.0
S18_1662,Planes,28285.0
S10_1949,Classic Cars,24998.0
S12_2823,Motorcycles,13961.0


As shown, the **Classic Cars** product line make up the majority of the top 10 most in-demand and low-stock products in this store. As such, products in this line should be monitored closely and re-stocked most often to optimize inventory needs, ensuring that when customers wish to order one of the most in-demand Classic Car models, they are able to receive their product in a timely manner.


### Customer Spending Analysis

With product inventory better optimized, I now consider customer spending. By determining which customers bring in the most and which bring in the least profit, the Store will be able to make more informed decisions in marketing to consumers and monitoring spending KPIs. For example, the Store will be able to target campaigns specifically to low-spending customers to drive engagement while implementing rewards and loyalty benefits to its high-spending customers to maintain their support.

To analyze customer spending, I begin by creating a CTE to determine how much each customer spends and the resulting profit they bring in to the Store over their lifetime orders, then querying the top five customers that spend the most.

In [7]:
%%sql

WITH
customer_profits AS (
SELECT o.customerNumber, ROUND(SUM(od.quantityOrdered * (od.priceEach - p.buyPrice)), 2) AS profit
  FROM orders AS o
  JOIN orderdetails AS od
    ON o.orderNumber = od.orderNumber
  JOIN products AS p
    ON od.productCode = p.productCode
 GROUP BY o.customerNumber
)

SELECT c.contactLastName, c.contactFirstName, c.city, c.country, '$' || customer_profits.profit AS profit
  FROM customers AS c
  JOIN customer_profits
    ON customer_profits.customerNumber = c.customerNumber
 ORDER BY customer_profits.profit DESC
 LIMIT 5

 * sqlite:///stores.db
Done.


contactLastName,contactFirstName,city,country,profit
Freyre,Diego,Madrid,Spain,$326519.66
Nelson,Susan,San Rafael,USA,$236769.39
Young,Jeff,NYC,USA,$72370.09
Ferguson,Peter,Melbourne,Australia,$70311.07
Labrune,Janine,Nantes,France,$60875.3


I can then use the same CTE to determine the five lowest-spending customers by ordering the profit column in ascending order.


In [8]:
%%sql

WITH
customer_profits AS (
SELECT o.customerNumber, ROUND(SUM(od.quantityOrdered * (od.priceEach - p.buyPrice)), 2) AS profit
  FROM orders AS o
  JOIN orderdetails AS od
    ON o.orderNumber = od.orderNumber
  JOIN products AS p
    ON od.productCode = p.productCode
 GROUP BY o.customerNumber
)

SELECT c.contactLastName, c.contactFirstName, c.city, c.country, '$' || customer_profits.profit AS profit
  FROM customers AS c
  JOIN customer_profits
    ON customer_profits.customerNumber = c.customerNumber
 ORDER BY customer_profits.profit ASC
 LIMIT 5

 * sqlite:///stores.db
Done.


contactLastName,contactFirstName,city,country,profit
Young,Mary,Glendale,USA,$2610.87
Taylor,Leslie,Brickhaven,USA,$6586.02
Ricotti,Franco,Milan,Italy,$9532.93
Schmitt,Carine,Nantes,France,$10063.8
Smith,Thomas,London,UK,$10868.04


Although the above tables show us which customers spend the least, they are not the most useful for the Store to communicate campaigns in an ordered way. To solve this, I use the `NTILE()` window ranking function to divide customers into five equal buckets based on their spending and labelling each bucket with a tag. Should the Store wish to send an email campaign, for example, they would be able to select all customers with a specific tag to drive more targeted promotions, offers, and rewards. This will ultimately personalize customers' shopping experience and improve the Store's sales.

In [9]:
%%sql

WITH
customer_profits AS (
SELECT o.customerNumber, ROUND(SUM(od.quantityOrdered * (od.priceEach - p.buyPrice)), 2) AS profit
  FROM orders AS o
  JOIN orderdetails AS od
    ON o.orderNumber = od.orderNumber
  JOIN products AS p
    ON od.productCode = p.productCode
 GROUP BY o.customerNumber
)

SELECT c.contactFirstName || ' ' || c.contactLastName AS full_name, '$' || customer_profits.profit AS profit, 
    
       CASE 
       WHEN NTILE(5) OVER(ORDER BY customer_profits.profit DESC) = 1 THEN 'VIP Customer'
       WHEN NTILE(5) OVER(ORDER BY customer_profits.profit DESC) = 2 THEN 'High-Spending Customer'
       WHEN NTILE(5) OVER(ORDER BY customer_profits.profit DESC) = 3 THEN 'Average-Spending Customer'
       WHEN NTILE(5) OVER(ORDER BY customer_profits.profit DESC) = 4 THEN 'Low-Spending Customer'
       WHEN NTILE(5) OVER(ORDER BY customer_profits.profit DESC) = 5 THEN 'Lowest-Spending Customer'
       END AS profit_group
    
  FROM customers AS c
  JOIN customer_profits
    ON customer_profits.customerNumber = c.customerNumber
 ORDER BY customer_profits.profit DESC


 * sqlite:///stores.db
Done.


full_name,profit,profit_group
Diego Freyre,$326519.66,VIP Customer
Susan Nelson,$236769.39,VIP Customer
Jeff Young,$72370.09,VIP Customer
Peter Ferguson,$70311.07,VIP Customer
Janine Labrune,$60875.3,VIP Customer
Eric Natividad,$60477.38,VIP Customer
Rachel Ashworth,$60095.86,VIP Customer
Mike Graham,$60013.99,VIP Customer
Kwai Lee,$58669.1,VIP Customer
Sue Frick,$55931.37,VIP Customer


### Average Customer Profits

Calculating the current average customer profit will serve as a useful KPI as the Store continues to optimize its ordering and sales and begins growing again.

In [10]:
%%sql

WITH
customer_profits AS (
SELECT o.customerNumber, SUM(od.quantityOrdered * (od.priceEach - p.buyPrice)) AS profit
  FROM orders AS o
  JOIN orderdetails AS od
    ON o.orderNumber = od.orderNumber
  JOIN products AS p
    ON od.productCode = p.productCode
 GROUP BY o.customerNumber
)

SELECT '$' || ROUND(AVG(customer_profits.profit), 2) AS average_customer_profits
  FROM customer_profits

 * sqlite:///stores.db
Done.


average_customer_profits
$39039.59


With an average customer profit of $39,039.59, the Store can also determine how much of the profit it can use to reinvest into the business. Monitoring this KPI as marketing campaigns targeting specific customer groups are made is a strong way to indicate whether business strategies are effective or not. Additionally, the average customer profits figure can be used to drive financial modelling and predictions: an increase of 5 customers over the next year, for example, should bring an additional 195,197.95 profit to the business over the customers' lifetime orders.

### Conclusion

The data analysis helped generate results answering the three essential questions. I determined that the Classic Cars product line was most likely to be low stock and high performing by calculating a priority index for each product. I also ranked customers into spending buckets and determined the important KPI of average customer profit over customers' order lifetimes. All this information will help the Store optimize inventory of popular products, tailor personalized marketing campaigns to specific customer brackets to drive growth, and monitor various KPIs to determine if new business strategies are being implemented efficiently and effectively.

Ultimately, the generated data will allow the Store to make data-informed decisions and reverse its decline amidst recent challenges.